In [1]:
import numpy as np
from lenstronomy.Cosmo.lens_cosmo import LensCosmo
"import comsmology parameter,cite Planck 2015 results XIII. Cosmological parameters,Ade et al.2016"
from astropy.cosmology import Planck15 as cosmo
"package in lenstronomy to calculate deflection angle in different lens models"
from lenstronomy.LensModel.Profiles.sie import SIE
"package in lenstronomy to convert units"
from lenstronomy.Util import param_util
"package to convert mass_200 to sigmav"
import m200tosigmav
""
import matplotlib.pyplot as plt

from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LensModel.Profiles.convergence import Convergence
from lenstronomy.LensModel.Profiles.shear import Shear
convergence = Convergence()
sie = SIE()
shear=Shear()
'reduced hubble constant'
h=np.array(cosmo.H(0))/100


ModuleNotFoundError: No module named 'lenstronomy'

In [2]:
def calmu_sis_lenstro(lenx_list,leny_list,lenz_list,lenm_list,e,phi,soutestz,windows,npix,ext_kappa,z_ex_kappa,ext_shear1,ext_shear2,z_ex_shear):
    xmin = -windows/2
    xmax = windows/2
    x=np.linspace(xmin,xmax,npix)
    len1rayx,len1rayy = np.meshgrid(x,x)
    redshift_list = []
    kwargs_lens = []
    lens_model_list = []
    n = len(lenx_list)
    q = 1 - e
    e1, e2 = param_util.phi_q2_ellipticity(phi=phi, q=q)
    for i in range(n):
        lens_cosmo = LensCosmo(z_lens=lenz_list[i], z_source=soutestz, cosmo=cosmo)
        theta_E = lens_cosmo.sis_sigma_v2theta_E(m200tosigmav.cal(lenm_list[i], lenz_list[i]))
        a = {'theta_E': theta_E, 'e1': e1[i], 'e2': e2[i], 'center_x': lenx_list[i], 'center_y': leny_list[i]}

        kwargs_lens.append(a)
        lens_model_list.append('SIE')
        redshift_list.append(lenz_list[i])
    redshift_list.append(z_ex_kappa)
    lens_model_list.append('CONVERGENCE')
    a = {'kappa':ext_kappa}
    kwargs_lens.append(a)
    redshift_list.append(z_ex_shear)
    lens_model_list.append('SHEAR')
    a = {'gamma1': ext_shear1, 'gamma2': ext_shear2}
    kwargs_lens.append(a)
    lensModel_mp = LensModel(lens_model_list=lens_model_list, z_source=soutestz, lens_redshift_list=redshift_list,
                             multi_plane=True)
    len2rayx, len2rayy=lensModel_mp.ray_shooting(x=len1rayx,y=len1rayy,kwargs=kwargs_lens)
    return len2rayx, len2rayy



In [ ]:

x1,y1=calmu_sis_lenstro(lenx_list,leny_list,lenz_list,lenm_list,1-q,phi,0.4,windows=20,npix=500,ext_kappa=0.01,z_ex_kappa=0.2,ext_shear1=0.01,ext_shear2=0.01,z_ex_shear=0.2)
x2,y2=calmu_sis_lenstro(lenx_list,leny_list,lenz_list,lenm_list,1-q,phi,1.0,windows=20,npix=500,ext_kappa=0.01,z_ex_kappa=0.2,ext_shear1=0.01,ext_shear2=0.01,z_ex_shear=0.2)
image_2d_src_plane_frist = sersic_2d(
    x=x1,
    y=y1,
    xcen=0,
    ycen=0,
    phi=0,
    axis_ratio=1,
    i_eff=1.0,
    r_eff=0.1,
    n=1.0
)
image_2d_src_plane_secend = sersic_2d(
    x=x2,
    y=y2,
    xcen=0,
    ycen=0,
    phi=0,
    axis_ratio=1,
    i_eff=1.0,
    r_eff=0.1,
    n=1.0
)



plt.imshow(image_2d_src_plane_frist+image_2d_src_plane_secend,cmap='jet', origin='lower')


plt.show()
